# Анализ данных и создание модели с предобработкой

In [ ]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('data/youtube_channel_real_performance_analytics.csv')

# Печать имен столбцов
print(data.columns)

Index(['ID', 'Video Duration', 'Video Publish Time', 'Days Since Publish',
       'Day', 'Month', 'Year', 'Day of Week', 'Revenue per 1000 Views (USD)',
       'Monetized Playbacks (Estimate)', 'Playback-Based CPM (USD)',
       'CPM (USD)', 'Ad Impressions', 'Estimated AdSense Revenue (USD)',
       'DoubleClick Revenue (USD)', 'YouTube Ads Revenue (USD)',
       'Watch Page Ads Revenue (USD)', 'YouTube Premium (USD)',
       'Transaction Revenue (USD)', 'Transactions',
       'Revenue from Transactions (USD)', 'Reactions', 'Chat Messages Count',
       'Reminders Set', 'Stream Hours', 'Remix Views', 'Remix Count',
       'Subscribers from Posts', 'New Comments', 'Shares', 'Like Rate (%)',
       'Dislikes', 'Likes', 'Unsubscribes', 'New Subscribers',
       'Returned Items (USD)', 'Unconfirmed Commissions (USD)',
       'Approved Commissions (USD)', 'Orders', 'Total Sales Volume (USD)',
       'End Screen Click-Through Rate (%)', 'End Screen Impressions',
       'End Screen Clicks', 

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
import plotly.express as px
import matplotlib.pyplot as plt

# Загрузка данных
data = pd.read_csv('/Users/purota/Downloads/youtube_analytics_project/youtube_channel_real_performance_analytics.csv')

# Убираем возможные пропуски
data = data.dropna()

# Преобразуем столбец 'Day of Week' в категориальный тип
data['Day of Week'] = data['Day of Week'].astype('category')

# Отбираем признаки для модели
X = data[['Subscribers', 'Video Duration', 'Day of Week']]  # с 'Day of Week'
y = data['Estimated Revenue (USD)']  # Целевая переменная

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабируем только числовые признаки
scaler = StandardScaler()
X_train_scaled = X_train[['Subscribers', 'Video Duration']]  # Только числовые столбцы
X_test_scaled = X_test[['Subscribers', 'Video Duration']]

# Масштабируем
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

# Воссоздаем данные с отмасштабированными признаками
X_train = X_train.copy()
X_test = X_test.copy()

X_train[['Subscribers', 'Video Duration']] = X_train_scaled
X_test[['Subscribers', 'Video Duration']] = X_test_scaled

# Обновляем категориальный столбец 'Day of Week'
X_train['Day of Week'] = X_train['Day of Week'].astype('category').cat.codes
X_test['Day of Week'] = X_test['Day of Week'].astype('category').cat.codes
print(X_train['Day of Week'])
# Создание и обучение модели CatBoost
model = CatBoostRegressor(iterations=1000, depth=7, learning_rate=0.05, loss_function='RMSE', cat_features=[2])
model.fit(X_train, y_train)

# Сохранение модели
model.save_model('catboost_model.cbm')
print("Модель успешно сохранена!")

# Оценка модели
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

# Выводим результаты
print(f'Обучение модели завершено. Результаты модели:')
print(f'Обучающая выборка: {train_score}')
print(f'Тестовая выборка: {test_score}')

210    3
339    4
299    0
75     4
84     6
      ..
71     2
106    5
270    6
348    5
102    3
Name: Day of Week, Length: 291, dtype: int8
0:	learn: 14.0678448	total: 3.09ms	remaining: 3.09s
1:	learn: 13.9077648	total: 4.2ms	remaining: 2.09s
2:	learn: 13.7074559	total: 5.22ms	remaining: 1.73s
3:	learn: 13.5657519	total: 6.38ms	remaining: 1.59s
4:	learn: 13.3995132	total: 6.97ms	remaining: 1.39s
5:	learn: 13.2839964	total: 7.83ms	remaining: 1.3s
6:	learn: 13.1864863	total: 8.54ms	remaining: 1.21s
7:	learn: 13.0442051	total: 9.43ms	remaining: 1.17s
8:	learn: 12.8841964	total: 10.2ms	remaining: 1.13s
9:	learn: 12.8040836	total: 11.2ms	remaining: 1.11s
10:	learn: 12.6828522	total: 12.1ms	remaining: 1.08s
11:	learn: 12.5440375	total: 13.1ms	remaining: 1.08s
12:	learn: 12.4336085	total: 13.8ms	remaining: 1.05s
13:	learn: 12.3959630	total: 13.9ms	remaining: 981ms
14:	learn: 12.3302547	total: 14.3ms	remaining: 939ms
15:	learn: 12.2254495	total: 15.5ms	remaining: 951ms
16:	learn: 12.1768713